In [1]:
#creating a dataframe
path="/home/affine/Expedia/Random Forest/RF Sample Data/AD_Region_Mar_1.csv"
AD_Region_Mar = sqlCtx.read.format('com.databricks.spark.csv').options(header='true', inferschema='true',delimiter=',').load(path)

In [2]:
from pyspark.sql.functions import *
import datetime
from pyspark.sql.types import StringType

def change_date(date):
    date = datetime.datetime.strptime(date, '%m/%d/%Y').strftime('%Y-%m-%d')
    return date

change_date_udf = udf(change_date, StringType())
#AD_Region_Mar_new=AD_Region_Mar.withColumn("activity_date_new",change_date_udf(AD_Region_Mar.activity_date))

#AD_Region_Mar_train = AD_Region_Mar_new.filter((AD_Region_Mar_new["activity_date_new"] >= "2015-01-01") & (AD_Region_Mar_new["activity_date_new"] <= "2015-01-27"))
#AD_Region_Mar_val = AD_Region_Mar_new.filter((AD_Region_Mar_new["activity_date_new"] < "2015-01-01") | (AD_Region_Mar_new["activity_date_new"] > "2015-01-27"))

AD_Region_Mar_train = AD_Region_Mar
AD_Region_Mar_val = AD_Region_Mar


In [3]:
#Aggregating the data to the required level
from pyspark.sql.functions import *
AD_Region_Mar_T = AD_Region_Mar_train.groupBy(AD_Region_Mar_train.bw_bucket,AD_Region_Mar_train.los_bucket \
                      ,AD_Region_Mar_train.hotel_type_bucket,AD_Region_Mar_train.star_rating_bucket,AD_Region_Mar_train.market_bucket \
                      ,AD_Region_Mar_train.seller_rank_bucket).sum("traffic","booking")

AD_Region_Mar_T1 = AD_Region_Mar_T.withColumnRenamed('sum(traffic)','Traffic')
AD_Region_Mar_T1 = AD_Region_Mar_T1.withColumnRenamed('sum(booking)','Booking')


AD_Region_Mar_V = AD_Region_Mar_val.groupBy(AD_Region_Mar_val.bw_bucket,AD_Region_Mar_val.los_bucket \
                      ,AD_Region_Mar_val.hotel_type_bucket,AD_Region_Mar_val.star_rating_bucket,AD_Region_Mar_val.market_bucket \
                      ,AD_Region_Mar_val.seller_rank_bucket).sum("traffic","booking")

AD_Region_Mar_V1 = AD_Region_Mar_V.withColumnRenamed('sum(traffic)','Traffic')
AD_Region_Mar_V1 = AD_Region_Mar_V1.withColumnRenamed('sum(booking)','Booking')

######Filtering out the Traffic (Outliers) from data
AD_Region_Mar_T1 = AD_Region_Mar_T1[~((AD_Region_Mar_T1.Traffic<=4) & (AD_Region_Mar_T1.Booking>0))]
AD_Region_Mar_T1 = AD_Region_Mar_T1[~((AD_Region_Mar_T1.Traffic==5) & (AD_Region_Mar_T1.Booking==2))]
AD_Region_Mar_T1 = AD_Region_Mar_T1[~((AD_Region_Mar_T1.Traffic==6) & (AD_Region_Mar_T1.Booking==2))]


In [4]:
import numpy

#####Converting the data type of required columns to factor
from pyspark.ml.feature import *
def indexStringColumns(df,cols):
    tempdf = df
    for col in cols:
        stringIndexer = StringIndexer(inputCol=col,outputCol=col+"-num")
        si_model = stringIndexer.fit(tempdf)
        tempdf = si_model.transform(tempdf)
    return tempdf
cols = {"bw_bucket","los_bucket","hotel_type_bucket","star_rating_bucket","market_bucket","seller_rank_bucket"}
dfnumeric = indexStringColumns(AD_Region_Mar_T1,cols)

In [5]:
def oneHotEncodeColumns(df,cols):
    tempdf = df
    for col in cols:
        onehotenc = OneHotEncoder(inputCol=col+"-num",outputCol=col+"-onehot")
        tempdf = onehotenc.transform(tempdf).drop(col+"-num")
    return tempdf
AD_Region_Mar_T1_Class = oneHotEncodeColumns(dfnumeric,{"bw_bucket","los_bucket","hotel_type_bucket","star_rating_bucket","market_bucket","seller_rank_bucket"})

In [6]:
#####Transforming the training dataset
#Defining function for the same
def genRows(row):
    traffic = row[6]
    booking = row[7]
    bookingcounter = 0
    rowList = []
    
    for i in range(traffic):
        row1 = row[:]                   
        #row1[6]=1
        row1= list(row1)
        row1[6]=1
        row1 = tuple(row1)
        if bookingcounter < int(booking):
            #row1[7]=1
            row1= list(row1)
            row1[7]=1
            row1 = tuple(row1)
        else :
            #row1[7]=0
            row1= list(row1)
            row1[7]=0
            row1 = tuple(row1)
        bookingcounter += 1
        rowList.append(row1)
    return rowList

#Applying function on the complete dataset (after converting to rdd)
rdd1 = AD_Region_Mar_T1_Class.rdd
rdd2 = rdd1.flatMap(lambda x : genRows(x))

In [7]:
#Reconverting to dataframe
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.mllib.linalg import SparseVector, VectorUDT


schema = StructType([StructField('bw_bucket', StringType(), True),
                     StructField('los_bucket', StringType(), True),
                     StructField('hotel_type_bucket', StringType(), True),
                     StructField('star_rating_bucket', StringType(), True),
                     StructField('market_bucket', StringType(), True),
                     StructField('seller_rank_bucket', StringType(), True),
                     StructField('Traffic', IntegerType(), True),
                     StructField('Booking', IntegerType(), True),
                     StructField('hotel_type_bucket-onehot', VectorUDT(), True),
                     StructField('bw_bucket-onehot',  VectorUDT(), True),
                     StructField('region_bucket-onehot',  VectorUDT(), True),
                     StructField('los_bucket-onehot',  VectorUDT(), True),
                     StructField('seller_rank_bucket-onehot',  VectorUDT(), True),
                     StructField('star_rating_bucket-onehot',  VectorUDT(), True)])
df7 = sqlContext.createDataFrame(rdd2, schema)

In [8]:
from pyspark.sql.functions import udf

#Converting label to double datatype
toDouble = udf(lambda x : float(x * 1.00) , DoubleType())
df8 = df7.withColumn('Booking' , toDouble(df7.Booking))
#df8.dtypes

#Taking only relevant columns as features
colList = df8.columns 
colList.remove('Booking')
colList.remove('Traffic')
colList.remove('bw_bucket')
colList.remove('los_bucket')
colList.remove('hotel_type_bucket')
colList.remove('star_rating_bucket')
colList.remove('market_bucket')
colList.remove('seller_rank_bucket')
vecAssembler = VectorAssembler(inputCols=colList,outputCol="features")
Train_AD = vecAssembler.transform(df8).select("Booking","Traffic","bw_bucket","los_bucket","hotel_type_bucket","star_rating_bucket","market_bucket","seller_rank_bucket","features").withColumnRenamed("Booking", "label")


In [9]:
#####Transforming the validation dataset
cols = {"bw_bucket","los_bucket","hotel_type_bucket","star_rating_bucket","market_bucket","seller_rank_bucket"}
dfnumeric_V1 = indexStringColumns(AD_Region_Mar_T1,cols)
#dfnumeric_V1.show()
df9 = oneHotEncodeColumns(dfnumeric_V1,{"bw_bucket","los_bucket","hotel_type_bucket","star_rating_bucket","market_bucket","seller_rank_bucket"})

col = df9.columns 
col.remove('Booking')
col.remove('Traffic')
col.remove('bw_bucket')
col.remove('los_bucket')
col.remove('hotel_type_bucket')
col.remove('star_rating_bucket')
col.remove('market_bucket')
col.remove('seller_rank_bucket')
vecAssembler = VectorAssembler(inputCols=col,outputCol="indexedFeatures")
Valid_DF = vecAssembler.transform(df9).select("Booking","Traffic","bw_bucket","los_bucket","hotel_type_bucket","star_rating_bucket","market_bucket","seller_rank_bucket","indexedFeatures")



In [10]:
####Random Forest model building
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import StringIndexer, VectorIndexer

labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(Train_AD)
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures").fit(Train_AD)
rf = RandomForestRegressor(labelCol="indexedLabel", featuresCol="indexedFeatures",featureSubsetStrategy="onethird",\
                           numTrees=1000,impurity="variance",maxBins=32)
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf])
model = pipeline.fit(Train_AD)
rfModel = model.stages[2]
print (rfModel)
####Prediction on validation dataset
predictions = rfModel.transform(Valid_DF)
#predictions.printSchema()


RandomForestRegressionModel (uid=rfr_f08998ff073d) with 1000 trees


In [12]:
def actual_conv(traffic,booking):
    return (booking/traffic)

actual_conv_udf = udf (actual_conv, DoubleType())

predictions = predictions.withColumn("actual_conv",actual_conv_udf(predictions.Traffic,predictions.Booking))
   

In [13]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(
    labelCol="actual_conv", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)

print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 0.0397409


In [14]:
df_final =predictions
df_final = df_final.withColumnRenamed("prediction","Predict_Py")
df_temp = df_final
df_final.describe('Predict_Py').show()

+-------+--------------------+
|summary|          Predict_Py|
+-------+--------------------+
|  count|               11951|
|   mean|0.024577660378729757|
| stddev|0.017992267318730507|
|    min|0.003466579230573777|
|    max| 0.09580963988275441|
+-------+--------------------+



In [15]:
df_final.toPandas().to_csv('NA_1000')

In [16]:
def error(actual,predict):
    x = actual-predict  
    
    if x < 0.0:
        x  = x*(-1)
    else : x
    
    if actual == 0.0:
        y = x
    else :
        y = x/actual
    return x

error_udf = udf(error,DoubleType())

df_final1 =  df_temp.withColumn("error_Py",error_udf(df_temp.actual_conv,df_temp.Predict_Py))
df_final1.describe('error_py').show()

+-------+--------------------+
|summary|            error_py|
+-------+--------------------+
|  count|               11951|
|   mean| 0.02714291305191647|
| stddev|0.029028840662426114|
|    min|3.931609115252060...|
|    max|  0.6606579178262031|
+-------+--------------------+



In [112]:
import pandas as pd

look_up = df_final1.toPandas()
look_up = look_up.round(2)

#bins = [0.00,0.01,0.02,0.03,0.04,0.05,0.10,0.30,0.50,0.90,1.00]
#bins1 = [0.00,0.10,0.20,0.30,0.40,0.50,0.60,0.70,0.80,0.90,1.00]

def buckets(x):
    if x <= 0.01:
        return 1
    elif (x > 0.01) and (x <= 0.02):
        return 2
    elif (x > 0.02) and (x <= 0.03):
        return 3
    elif (x > 0.03) and (x <= 0.04):
        return 4
    elif (x > 0.04) and (x <= 0.05):
        return 5
    elif (x > 0.05) and (x <= 0.10):
        return 6
    elif (x > 0.10) and (x <= 0.30):
        return 7
    elif (x > 0.30) and (x <= 0.50):
        return 8
    elif (x > 0.50) and (x <= 0.90):
        return 9
    elif x > 0.90:
        return 10
    

#look_up['error_bucket']  = pd.cut(look_up['error_Py'], bins)
#look_up['actual_bucket']  = pd.cut(look_up['actual_conv'], bins)


Total_Traffic = look_up.Traffic.sum(axis=0)

def per(Traffic):
    return (Traffic*100/Total_Traffic)

look_up["%_Traffic"] = look_up.Traffic.map(per)
look_up["error_bucket"] = look_up.error_Py.map(buckets)
look_up["actual_bucket"] = look_up.actual_conv.map(buckets)


In [122]:
x = look_up.groupby(['error_bucket'], as_index=False).agg({'%_Traffic':'sum'})
x.to_csv('error_NA_1000')
x

error_bucket  %_Traffic
0             1  33.345040
1             2  17.391668
2             3  16.351412
3             4   7.121235
4             5   9.067951
5             6  12.260633
6             7   4.358370
7             8   0.091984
8             9   0.011707

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(look_up['actual_conv'], look_up['Predict_Py'])
plt.show()

In [125]:
y = look_up.groupby(['actual_bucket'], as_index=False).agg({'%_Traffic':'sum'})
y.to_csv('actual_NA_1000')
y

actual_bucket  %_Traffic
0              1  60.833208
1              2   1.588815
2              3   2.475206
3              4   3.236165
4              5   3.749603
5              6  15.179034
6              7  12.819226
7              8   0.107036
8              9   0.011707

In [126]:
import numpy as np
table = pd.pivot_table(look_up, values='%_Traffic', index=['error_bucket'],columns=['actual_bucket'], aggfunc=np.sum)
table.to_csv('crosstab_NA_1000')
table

actual_bucket          1         2         3         4         5         6  \
error_bucket                                                                 
1              24.034586  0.446541  0.958306  1.117188  1.888181  4.900239   
2              10.977874  0.078605  1.043600  1.433278  1.180740  2.446775   
3              11.954577  0.180623  0.150519  0.468282  0.332815  2.726072   
4               2.704330  0.727510  0.220762  0.043483  0.347867  2.356463   
5               6.382018  0.155537  0.102019  0.173933       NaN  1.077049   
6               4.779824       NaN       NaN       NaN       NaN  1.672437   
7                    NaN       NaN       NaN       NaN       NaN       NaN   
8                    NaN       NaN       NaN       NaN       NaN       NaN   
9                    NaN       NaN       NaN       NaN       NaN       NaN   

actual_bucket         7         8         9  
error_bucket                                 
1                   NaN       NaN       NaN  
2              0.230796       NaN       NaN  
3              0.538525       NaN       NaN  
4              0.720820       NaN       NaN  
5              1.177395       NaN       NaN  
6              5.808372       NaN       NaN  
7              4.343318  0.015052       NaN  
8                   NaN  0.091984       NaN  
9                   NaN       NaN  0.011707

In [68]:
####histograms

import matplotlib.pyplot as plt
from numpy.random import normal

data = look_up['actual_conv'].as_matrix()
plt.hist(data, bins=10)  ##normed= True,  cumulative=True
plt.title("actual_conv")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.show()

In [75]:
###scatter plot

import matplotlib.pyplot as plt
plt.scatter(look_up['actual_conv'], look_up['Predict_Py'])
plt.show()

In [197]:
import matplotlib.pyplot as plt
from numpy.random import normal

data = look_up['Traffic'].as_matrix()
plt.hist(data, bins=100)  ##normed= True,  cumulative=True
plt.title("Traffic")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.show()

In [82]:
###scatter plot

import matplotlib.pyplot as plt
plt.scatter(look_up['Booking'], look_up['Predict_Py'])
plt.show()

In [83]:
###scatter plot

import matplotlib.pyplot as plt
plt.scatter(look_up['Booking'], look_up['Traffic'])
plt.show()